# Institute for Behavioral Genetics International Statistical Genetics 2023 Workshop 

## Genetic relatedness exploration with Hail

In this practical, we will learn how to:

1) Simulate random mating in Hail

2) Run various relatedness estimation methods in Hail

3) Use Hail to investigate how these methods work on a structured and admixed dataset

# 1. Import and initialize Hail

We import Hail, initialize it, and import some plotting tools as well.

In [ ]:
import hail as hl
hl.init()
from hail.plot import output_notebook, show
from bokeh.models import Slope
output_notebook()

# 2. Read HGDP data, add population information, and run PCA

This bit is a sped-up review from the first notebook. We want to have the principal component plot as reference for relatedness exploration below!

In [ ]:
mt = hl.read_matrix_table('resources/hgdp.mt')
sd = hl.import_table('resources/HGDP_sample_data.tsv',
                     key='sample_id',
                     impute=True)
mt = mt.annotate_cols(sample_idx = hl.int(hl.scan.count()), sample_data = sd[mt.s])

In [ ]:
_, scores, _ = hl.hwe_normalized_pca(mt.GT)

In [ ]:
show(hl.plot.scatter(scores.scores[0], scores.scores[1], 
                     label=mt.index_cols(scores.key).sample_data.continental_pop,
                     size=8,
                     xlabel='PC1',
                     ylabel='PC2'))

# 3. Simulate 3 generations of random mating.

Hail contains a simple random mating simulator that can be a great tool for exploring relatedness and admixture in populations.

The starting dataset is highly **structured** but less **admixed**. We can see distinct ancestral clusters with spaces  between. If we were to make the same plot of a random slice of the human population, the clusters would be connected by individuals falling on the clines between.

Here we simulate three rounds of random mating, where each generation creates a number of pairs equal to half its generation size (with replacement, but no self-pairs), and each pair has two children. Each pair is within a single generation; pairs do not span generations. There are no restrictions on pair selection other than that a sample cannot pair with itself.

In [ ]:
mt1 = hl.simulate_random_mating(mt,
                                pairs_per_generation_multiplier=0.5,
                                children_per_pair=2,
                                n_rounds=3,
                                seed=5).key_cols_by('sample_idx')
mt1 = mt1.annotate_cols(**mt.key_cols_by('sample_idx').index_cols(mt1.sample_idx).sample_data).persist()
mt1.count()

In [ ]:
_, scores1, _ = hl.hwe_normalized_pca(mt1.GT)

#### Take a few moments to explore and discuss the PC plot of the new dataset, and where the new samples fall. What structure is present?

In [ ]:
joined_data = mt1.index_cols(scores1.key)
show(hl.plot.scatter(scores1.scores[0], scores1.scores[1],
                     label=joined_data.continental_pop,
                     hover_fields={'sample_id': joined_data.s},
                     size=6,
                     xlabel='PC1',
                     ylabel='PC2'))

# 4. Relatedness estimation

Estimation of relatedness between individuals is a core tool in statistical genetics. It is commonly used to verify reported relatedness (both false negatives and false positives), and is an important input to statistical analysis.

Some statistical analyses (for instance, GWAS using a simple linear regression) are built around the assumption that variance in genotypes and phenotypes are independent and identically distributed, and an estimated relatedness graph can be used to filter samples to an unrelated set before running these methods.

Other methods, like SAIGE, explicitly account for sample covariance to produce statistically sound effect estimation for datasets with related samples.

## Coefficient of kinship versus coefficient of relationship.

The kinship coefficient estimated in methods below is defined as the probability that two homologous alleles drawn from each of two individuals are identical by descent. The similar "coefficient of relationship", defined as the fraction of genetic material shared identically-by-descent, is equal to twice the kinship coefficient.

| Relationship | Kinship Coefficient ($\phi$) | Coefficient of Relationship ($r$) |
| :--- | --- | --- |
| Self | 1/2 = 0.5 | 1.0 |
| Parent-Child | ??? | ??? |
| Full Sibling | ??? | ??? |
| Grandparent-Grandchild | 1/8 = 0.125 | 0.25 |
| Avuncular Pair | ??? | ??? |
| First Cousin | ??? | ??? |

### Exercise

Fill in thet missing entries in the above table. Double clicking on a markdown cell, like this one, allows you to edit it.

# 5. Identity By Descent

All of the models we explore today are so-called "SNP-based" methods. These methods operate in two phases. In phase one, each variant is independently used as an indicator of the relatedness of a pair of samples. In phase two, the kinship of a pair of samples is defined as a weighted sum of these independent tests. The three methods we explore today: IBD, KING, and PC-Relate, differ only in their choice of weights!

The PLINK or "identity-by-descent" estimator of kinship is defined in Purcell, et al. 2007. The definition is rather complex so we do not repeat it here. Instead, consider an equivalent formulation presented Manichaikul, et al. 2010:

Let
- $X_{im}$ be the number of alternate alleles in the $i$-th sample's genotype at the $m$-th biallelic SNP.
- $p_s$ be the population alternate allele frequency of the $m$-th biallelic SNP.
- $M$ is the number of biallelic SNPs.

The estimator for the kinship between sample $i$ and sample $j$ is given by:

$$
\widehat{\phi_{ij}} = \frac{1}{2} - \frac{\sum_{m} (X_{im} - X_{jm})^2}{\sum_{m} 8 p_{m} (p_{m} - 1)}
$$

Intuitively: we are counting down from an expectation of monozygotic twins ($\frac{1}{2}$). If two individuals share all their genotypes, the numerator of the second term goes to zero and $\widehat{\phi_{ij}}$ goes to $\frac{1}{2}$. If two individuals have opposing homozygous genotypes at every site, the numerator goes to $4M$ and the denominator goes to $8M$ (note that the variance tends one if every site tends to an even split between homozygous alternate and homozygous reference).

Below we use the relatedness graph produced by `simulate_random_mating` to look up the relatedness for a given pair, and adds a little bit of noise to make the plots look more realistic.

In [ ]:
rel = mt1.index_globals().relatedness
def get_rel(s1, s2):
    return (hl.case()
            .when(s1 == s2, 0.5)
            .when(s1 > s2, hl.coalesce(rel.get(s1).get(s2), 0.0))
            .default(hl.coalesce(rel.get(s2).get(s1), 0.0))) + hl.rand_norm(0, 0.005) # jitter

In [ ]:
ibd = hl.identity_by_descent(mt1.key_cols_by(sample_idx=hl.str(mt1.sample_idx)), min=0, max=1)
ibd = ibd.key_by(i=hl.int(ibd.i), j=hl.int(ibd.j))
fig = hl.plot.scatter(get_rel(ibd.i, ibd.j) * 2,
                      ibd.ibd.PI_HAT, xlabel='True coefficient of relatedness', ylabel='IBD PI_HAT',
                      hover_fields={'id1': ibd.i, 'id2': ibd.j})
fig.add_layout(Slope(gradient=1, y_intercept=0, line_color='red', line_dash='dashed'))
show(fig)

### Exercise: discuss -- How does the estimate deviate from true relatedness? Why?


# 6. The KING estimator

The estimator for kinship given in Section 5 assumes a single homogenous population. Do you know why?

The KING estimator for kinship relaxes this assumption to multiple homogenous populations with no admixture. The estimator was defined by Manichaikul et al. 2010. We present here only the "between-family" estimator, which is the only one implemented in Hail.

Let

- $N^{Aa}_i$ be the number of heterozygote genotypes in sample $i$.

The KING estimator for between-family kinship is given by:

$$
\widehat{\phi^{\textrm{between}}_{ij}} = \frac{1}{2} - \frac{\sum_m (X_{im} - X_{jm})^2}{4\textrm{min}(N^{Aa}_i, N^{Aa}_j)}
$$

Intuitively: we are again counting down from an assumption of monozygotic twins. Critically, instead of a sum of the binomial variance for two draws (one for each allele), $2p_m(p_m - 1)$, Manichaikul et al. propose using the minimum count of heterozygous variants between the two samples. Manichaikul et al. justify this proposal by noting that the fraction of heterozygotic genotypes is equal to the expectation of twice the binomial variance of alternate allele frequency:

$$
N^{Aa}_i / M = \mathbb{E}_m( 2p_m(1 - p_m) )
$$

And they justify using the minimum of the two samples' heterozygote counts thusly:

> In order to guard against potential estimation inflation due to departure from individual-level HWE, we consider the smaller of the observed heterozygosity rates [...]

In [ ]:
king = hl.king(mt1.GT).entries()
king = king.filter(king.sample_idx != king.sample_idx_1) # remove self-comparisons

Now we'll make a plot!

In [ ]:
fig = hl.plot.scatter(get_rel(king.sample_idx, king.sample_idx_1),
                      king.phi, 
                      xlabel='True kinship', 
                      ylabel='King phi (estimated kinship)',
                      hover_fields={'id1': king.sample_idx, 'id2': king.sample_idx_1})

fig.add_layout(Slope(gradient=1, y_intercept=0, line_color='red', line_dash='dashed'))
show(fig)

### Exercise: discuss -- How does the estimate deviate from true relatedness? Why?

# 7. The PC-Relate Estimator

When an individual has admixed ancestry, the KING model breaks down. Alleles that are rare in all but one ancestry group are treated as very informative to relatedness. However, these alleles are simply markers of the ancestry group. The PC-Relate method corrects for this.

PC-Relate slightly modifies the usual estimator for relatedness: occurrences of population allele frequency are replaced with an "individual-specific allele frequency". This modification allows the method to correctly weight an allele according to an individual’s unique ancestry profile.



Let 
- $\widehat{\mu_{im}}$ be the "individual-specific allele frequency" at a given genetic locus. Define it as one-half the best linear estimate of an individual's genotype using only the provided principal components.
- $\widehat{\sigma^2_{im}} = \widehat{\mu_{im}}(1 - \widehat{\mu_{im}})$
- $\widehat{\sigma_{im}} = \sqrt{\widehat{\sigma^2_{im}}}$

The PC-Relate estimator of kinship is then defined as:

$$
\widehat{\phi_{ij}} = \frac{\sum_m (g_{im} - 2\mu_{im}) (g_{jm} - 2\mu_{jm})}{4 \sum_m \widehat{\sigma_{im}} \widehat{\sigma_{jm}}}
$$

Intuitively: calculate the covariance of the residual genotypes (after "individual-specific mean" centering) normalized by the sum of their "individual-specific allele frequency variances".

In [ ]:
%%time
pcrel = hl.pc_relate(mt1.GT, 0.01, k=2, statistics='kin')

In [ ]:
pcrel = pcrel.key_by(i=pcrel.i.sample_idx, j=pcrel.j.sample_idx)
fig = hl.plot.scatter(get_rel(pcrel.i, pcrel.j),
                      pcrel.kin, xlabel='True kinship', ylabel='PC-Relate Kinship',
                      hover_fields={'id1': pcrel.i, 'id2': pcrel.j})
fig.add_layout(Slope(gradient=1, y_intercept=0, line_color='red', line_dash='dashed'))
show(fig)

In [ ]:
pcrel.describe()

In [ ]:
fig = hl.plot.scatter(get_rel(pcrel.i, pcrel.j),
                      pcrel.kin, xlabel='True kinship', ylabel='PC-Relate Kinship',
                      hover_fields={'id1': pcrel.i, 'id2': pcrel.j})
fig.add_layout(Slope(gradient=1, y_intercept=0, line_color='red', line_dash='dashed'))
show(fig)

### Exercise: detective work. Investigate the relationship between the individuals with kinship coefficient ~0.375.

Some useful information is encoded in the column fields of `mt`. You can show specific samples by editing the below code.
    
If you've got time after finishing this, do the same for a pair in the cluster with relatedness ~0.185!

To get you started, here's an example of how to interrogate three random samples, sample 50, 100, and 1000.

In [ ]:
mt1.filter_cols(hl.literal([50, 100, 1000]).contains(mt1.sample_idx)).cols().show()

### Exercise: model interrogation. 

PC-Relate uses an explicit `k` term to control how many principal components are used to in individual allele frequency predictions. Your job is to rerun the above, starting from the **PC-Relate** header, with various k values to interrogate how the number of principal components included affects the results.

What **k** seems best? What happens when **k** is small? Large?
